In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()
spark.version


'3.0.0'

In [2]:
%env SPARK_VERSION=3.0.0


env: SPARK_VERSION=3.0.0


In [3]:
pip install pydeequ

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pretty_html_table

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pydeequ

from pyspark.sql import SparkSession, Row

# TODO create spark session with jdbc driver path

In [4]:
spark = (SparkSession\
    .builder\
    .master("local[*]")\
    .appName("Connection through JDBC")\
    .config("spark.driver.extraClassPath","/home/jovyan/work/sqljdbc42.jar:/home/jovyan/work/deequ-1.2.2-spark-3.0.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())
print(pydeequ.deequ_maven_coord)
print(pydeequ.f2j_maven_coord)

com.amazon.deequ:deequ:1.2.2-spark-3.0
net.sourceforge.f2j:arpack_combined_all


In [5]:
server_url = "jdbc:sqlserver://192.168.1.129:1433" # think what should be here
database_name = "TRN"
url = server_url + ";" + "databaseName=" + database_name + ";"

table = "hr.employees"
user = "TestDQA" # your user creds here
password  = "TestDQA" # your user creds here

df=spark.read \
    .format("jdbc") \
    .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url",f"jdbc:sqlserver://192.168.1.129:1433;databaseName={database_name}") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .load()

df.show(10)
# TODO connect to DB with Spark using JDBC connection to read the data

+-----------+----------+---------+--------------------+------------+----------+------+--------+----------+-------------+
|employee_id|first_name|last_name|               email|phone_number| hire_date|job_id|  salary|manager_id|department_id|
+-----------+----------+---------+--------------------+------------+----------+------+--------+----------+-------------+
|        100|    Steven|     King|steven.king@sqltu...|515.123.4567|1987-06-17|     4|24000.00|      null|            9|
|        101|     Neena|  Kochhar|neena.kochhar@sql...|515.123.4568|1989-09-21|     5|17000.00|       100|            9|
|        102|       Lex|  De Haan|lex.de haan@sqltu...|515.123.4569|1993-01-13|     5|17000.00|       100|            9|
|        103| Alexander|   Hunold|alexander.hunold@...|590.423.4567|1990-01-03|     9| 9000.00|       102|            6|
|        104|     Bruce|    Ernst|bruce.ernst@sqltu...|590.423.4568|1991-05-21|     9| 6000.00|       103|            6|
|        105|     David|   Austi

In [6]:
from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("b")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+----+-----+
| entity|instance|name|value|
+-------+--------+----+-----+
|Dataset|       *|Size| 40.0|
+-------+--------+----+-----+



In [10]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: first_name: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 36,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 40
    },
    "histogram": [
        [
            "Den",
            1,
            0.025
        ],
        [
            "John",
            2,
            0.05
        ],
        [
            "Karen",
            2,
            0.05
        ],
        [
            "William",
            1,
            0.025
        ],
        [
            "Alexander",
            2,
            0.05
        ],
        [
            "Pat",
            1,
            0.025
        ],
        [
            "Lex",
            1,
            0.025
        ],
        [
            "Nancy",
            1,
            0.025
        ],
        [
            "Shelli",
            1,
            0.025
        ],
  

In [11]:
pip show pydeequ

Name: pydeequ
Version: 1.0.1
Summary: PyDeequ - Unit Tests for Data
Home-page: https://pydeequ.readthedocs.io
Author: Calvin Wang
Author-email: calviwan@amazon.com
License: Apache-2.0
Location: /opt/conda/lib/python3.10/site-packages
Requires: numpy, pandas
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
from pydeequ.suggestions import *
suggestionResult = ConstraintSuggestionRunner(spark) \
    .onData(df) \
    .addConstraintRule(DEFAULT()) \
    .run()
# Constraint Suggestions in JSON format
print(suggestionResult)


{'constraint_suggestions': [{'constraint_name': 'CompletenessConstraint(Completeness(first_name,None))', 'column_name': 'first_name', 'current_value': 'Completeness: 1.0', 'description': "'first_name' is not null", 'suggesting_rule': 'CompleteIfCompleteRule()', 'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint', 'code_for_constraint': '.isComplete("first_name")'}, {'constraint_name': 'CompletenessConstraint(Completeness(employee_id,None))', 'column_name': 'employee_id', 'current_value': 'Completeness: 1.0', 'description': "'employee_id' is not null", 'suggesting_rule': 'CompleteIfCompleteRule()', 'rule_description': 'If a column is complete in the sample, we suggest a NOT NULL constraint', 'code_for_constraint': '.isComplete("employee_id")'}, {'constraint_name': "ComplianceConstraint(Compliance('employee_id' has no negative values,employee_id >= 0,None))", 'column_name': 'employee_id', 'current_value': 'Minimum: 100.0', 'description': "'employ

In [18]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 3) \
        .hasMin("b", lambda x: x == 0) \
        .isComplete("c")  \
        .isUnique("a")  \
        .isContainedIn("a", ["foo", "bar", "baz"]) \
        .isNonNegative("b")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+------------+-----------+------------+--------------------+-----------------+--------------------+
|       check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+------------+-----------+------------+--------------------+-----------------+--------------------+
|Review Check|    Warning|     Warning|SizeConstraint(Si...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|MinimumConstraint...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|CompletenessConst...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|UniquenessConstra...|          Failure|Input data does n...|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|org.apache.spark....|
|Review Check|    Warning|     Warning|ComplianceConstra...|          Failure|org.apache.spark....|
+------------+-----------+------------+--------------------+-----------------+--------------------+
